In [1]:
import numpy as np
import pandas as pd
from keras.layers import LSTM , Dense ,Dropout
from keras.models import Sequential

Using TensorFlow backend.


In [2]:
train = pd.read_csv('train.csv')

In [3]:
test = pd.read_csv('test.csv')

In [4]:
train_dcast = pd.crosstab(index=[train['PID']], columns=train['Event'])
train_dcast.reset_index(drop=False, inplace=True)

In [5]:
train_dcast.head()

Event,PID,0001,0010,0014,0030,0042,0059,0066,0085,0088,...,V870,V872,V873,V874,V880,V890,V901,V908,V909,V910
0,1000001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1000011,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1000019,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,1000025,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1000029,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
submit = train_dcast.loc[:,train_dcast.columns != 'PID'].apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:10].index, index=['Event'+str(x) for x in range(1,11)]),axis=1).reset_index()
submit.drop('index',inplace=True, axis=1)

In [7]:
# setting column names in sequence
submit['PID'] = train_dcast['PID']
cols = submit.columns.tolist()
cols = cols[-1:] + cols[:-1]
submit = submit[cols]

In [8]:
submit.head()

,PID,Event1,Event2,Event3,Event4,Event5,Event6,Event7,Event8,Event9,Event10
0,1000001,3074,3263,2533,2632,2178,3041,9921,3272,3016,2582
1,1000011,9921,3106,8470,8100,9928,8502,2335,9711,7231,4097
2,1000019,7682,9928,7681,7680,6558,V283,7194,9920,6522,V270
3,1000025,3254,3641,2635,2674,3419,3410,5852,8444,9921,8443
4,1000029,2533,2773,3638,9921,4011,3051,8830,7373,3616,V723


In [9]:
x = train_dcast['PID']

In [10]:
y = train_dcast.drop(['PID'],axis = 1)

In [11]:
from sklearn.preprocessing import MinMaxScaler

In [12]:
sc = MinMaxScaler()

In [13]:
#x_train = sc.fit_transform(x)
x_train=x

In [14]:
x_train = np.reshape(x_train,(3000,1,1))

/home/anirudh/anaconda/lib/python2.7/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


In [23]:
y_train = sc.fit_transform(y)
#y_train=y

In [24]:
y_train.shape

(3000, 6472)

In [25]:
x_train.shape

(3000, 1, 1)

In [26]:
x_train

array([[[1000001]],

       [[1000011]],

       [[1000019]],

       ..., 
       [[1031882]],

       [[1031899]],

       [[1031900]]])

In [27]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import tensorflow as tf
from keras.layers import Dense,LSTM
from keras.models import Sequential

In [28]:
reg = Sequential()

In [29]:
reg.add(LSTM(units =8,activation = 'sigmoid',input_shape = (None , 1)))

In [ ]:
reg.add(Dense(units = 6472,kernel_initializer = 'uniform'))
reg.add(Dense(units = 6472 , kernel_initializer = 'uniform'))
reg.add(Dense(units=6472,kernel_initializer='uniform'))
reg.compile(optimizer='adam',loss='mean_squared_error')
reg.fit(x_train,y_train,batch_size = 50,epochs = 10)
test = pd.read_csv('test.csv')

Epoch 1/10
3000/3000 [==============================] - 16s - loss: 0.0047    
Epoch 2/10
1500/3000 [==============>...............] - ETA: 7s - loss: 0.0011

In [ ]:
y_pred = reg.predict(x_train,batch_size=50)

In [ ]:
y_pred.shape

In [ ]:
#y_pred = sc.inverse_transform(y_pred)

In [ ]:
y_pred

In [ ]:
pred = pd.DataFrame(data=y_pred)
submit_pred = pred.loc[:,pred.columns != 'PID'].apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:10].index, index=['Event'+str(x) for x in range(1,11)]),axis=1).reset_index()
submit_pred.drop('index',inplace=True, axis=1)


In [ ]:
# setting column names in sequence
submit_pred['PID'] = train_dcast['PID']
cols = submit_pred.columns.tolist()
cols = cols[-1:] + cols[:-1]
submit_pred = submit_pred[cols]

In [ ]:
submit.head()